In [3]:
%AddDeps org.joda joda-money 0.11 --transitive 

Marking org.joda:joda-money:0.11 for download
Obtained 2 files


In [2]:
%AddDeps com.databricks spark-xml_2.11 0.5.0 --transitive 

Marking com.databricks:spark-xml_2.11:0.5.0 for download
Obtained 2 files


In [1]:
%AddDeps mysql mysql-connector-java 5.1.47 --transitive 

Marking mysql:mysql-connector-java:5.1.47 for download
Obtained 2 files


In [7]:
import java.io._
print(new File(".").getAbsolutePath())
val whereami = System.getProperty("user.dir")
print(whereami)

/Users/xiaofeiwu/jcloud/assets/langs/workspace/spark/workspace/notebook/scala/./Users/xiaofeiwu/jcloud/assets/langs/workspace/spark/workspace/notebook/scala

whereami = /Users/xiaofeiwu/jcloud/assets/langs/workspace/spark/workspace/notebook/scala


/Users/xiaofeiwu/jcloud/assets/langs/workspace/spark/workspace/notebook/scala

In [9]:
import org.apache.spark.sql.SparkSession
import com.databricks.spark.xml._

val xmlfile="../../data/books.xml"
print(new File(xmlfile).exists())

val spark = SparkSession.builder.getOrCreate()
val df = spark.read
  .option("rowTag", "book")
  .xml(xmlfile)

val selectedData = df.select("author", "_id")
selectedData.write
  .option("rootTag", "books")
  .option("rowTag", "book")
  .xml("newbooks.xml")

true

xmlfile = ../../data/books.xml
spark = org.apache.spark.sql.SparkSession@79e57ed6
df = [_id: string, author: string ... 5 more fields]
selectedData = [author: string, _id: string]


[author: string, _id: string]

In [14]:
import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.types.{StructType, StructField, StringType, DoubleType}
import com.databricks.spark.xml._

val spark = SparkSession.builder.getOrCreate()
val customSchema = StructType(Array(
  StructField("_id", StringType, nullable = true),
  StructField("author", StringType, nullable = true),
  StructField("description", StringType, nullable = true),
  StructField("genre", StringType, nullable = true),
  StructField("price", DoubleType, nullable = true),
  StructField("publish_date", StringType, nullable = true),
  StructField("title", StringType, nullable = true)))


val df = spark.read
  .option("rowTag", "book")
  .schema(customSchema)
  .xml(xmlfile)

val selectedData = df.select("author", "title", "_id")
selectedData.write
  .option("rootTag", "books")
  .option("rowTag", "book")
  .xml("newbooks_2.xml")

spark = org.apache.spark.sql.SparkSession@79e57ed6
customSchema = StructType(StructField(_id,StringType,true), StructField(author,StringType,true), StructField(description,StringType,true), StructField(genre,StringType,true), StructField(price,DoubleType,true), StructField(publish_date,StringType,true), StructField(title,StringType,true))
df = [_id: string, author: string ... 5 more fields]
selectedData = [author: string, title: string ... 1 more field]


lastException: Throwable = null


[author: string, title: string ... 1 more field]

In [15]:
case class DFRecord(key: String, value: Int)
val sqlc = spark
import sqlc.implicits._
val df = sc.parallelize(1 to 10).map(x => DFRecord(x.toString, x)).toDF()

defined class DFRecord
sqlc = org.apache.spark.sql.SparkSession@79e57ed6
df = [key: string, value: int]


<console>:6: error: Symbol 'type scala.AnyRef' is missing from the classpath.
This symbol is required by 'class org.apache.spark.sql.catalyst.QualifiedTableName'.
Make sure that type AnyRef is in your classpath and check for conflicting dependencies with `-Ylog-classpath`.
A full rebuild may help if 'QualifiedTableName.class' was compiled against an incompatible version of scala.
  lazy val $print: String =  {
           ^


[key: string, value: int]

In [16]:
%%dataframe
df

key,value
1,1
2,2
3,3
4,4
5,5
6,6
7,7
8,8
9,9
10,10


In [17]:
%%dataframe --limit=3
df

key,value
1,1
2,2
3,3


In [19]:
val sqlc = spark
import sqlc.implicits._
case class Record(key: String, value: Int)
val df = sc.parallelize(1 to 10).map(x => Record(x.toString, x)).toDF()
df.registerTempTable("MYTABLE")

sqlc = org.apache.spark.sql.SparkSession@79e57ed6
defined class Record
df = [key: string, value: int]


[key: string, value: int]

In [20]:
%%SQL
SELECT * FROM MYTABLE WHERE value >= 6

+---+-----+
|key|value|
+---+-----+
|  6|    6|
|  7|    7|
|  8|    8|
|  9|    9|
| 10|   10|
+---+-----+



In [21]:
%%SQL
SELECT * FROM MYTABLE WHERE value >= 4

+---+-----+
|key|value|
+---+-----+
|  4|    4|
|  5|    5|
|  6|    6|
|  7|    7|
|  8|    8|
|  9|    9|
| 10|   10|
+---+-----+



In [6]:
import org.apache.spark.sql.SQLContext
// val sqlcontext = new org.apache.spark.sql.SQLContext(sc)
val sqlcontext = SparkSession.builder.getOrCreate()
val dataframe_mysql = sqlcontext.read.format("jdbc").option("url", "jdbc:mysql://127.0.0.1:4000/ofbiz").option("driver", "com.mysql.jdbc.Driver").option("dbtable", "PERSON").option("user", "ofbiz").option("password", "ofbiz").load()
// dataframe_mysql.show()

Sat Jan 26 18:09:57 CST 2019 WARN: Establishing SSL connection without server's identity verification is not recommended. According to MySQL 5.5.45+, 5.6.26+ and 5.7.6+ requirements SSL connection must be established by default if explicit option isn't set. For compliance with existing applications not using SSL the verifyServerCertificate property is set to 'false'. You need either to explicitly disable SSL by setting useSSL=false, or set useSSL=true and provide truststore for server certificate verification.


sqlcontext = org.apache.spark.sql.SparkSession@6116fea3
dataframe_mysql = [PARTY_ID: string, SALUTATION: string ... 34 more fields]


[PARTY_ID: string, SALUTATION: string ... 34 more fields]

In [10]:
%%dataframe --limit=5
dataframe_mysql

PARTY_ID,SALUTATION,FIRST_NAME,MIDDLE_NAME,LAST_NAME,PERSONAL_TITLE,SUFFIX,NICKNAME,FIRST_NAME_LOCAL,MIDDLE_NAME_LOCAL,LAST_NAME_LOCAL,OTHER_LOCAL,MEMBER_ID,GENDER,BIRTH_DATE,DECEASED_DATE,HEIGHT,WEIGHT,MOTHERS_MAIDEN_NAME,MARITAL_STATUS,SOCIAL_SECURITY_NUMBER,PASSPORT_NUMBER,PASSPORT_EXPIRE_DATE,TOTAL_YEARS_WORK_EXPERIENCE,COMMENTS,EMPLOYMENT_STATUS_ENUM_ID,RESIDENCE_STATUS_ENUM_ID,OCCUPATION,YEARS_WITH_EMPLOYER,MONTHS_WITH_EMPLOYER,EXISTING_CUSTOMER,CARD_ID,LAST_UPDATED_STAMP,LAST_UPDATED_TX_STAMP,CREATED_STAMP,CREATED_TX_STAMP
system,null,System,null,Account,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,2019-01-26 17:44:14.907,2019-01-26 17:44:14.843,2019-01-26 17:44:14.907,2019-01-26 17:44:14.843
_NA_,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,2019-01-26 17:45:08.391,2019-01-26 17:45:08.362,2019-01-26 17:45:08.391,2019-01-26 17:45:08.362
admin,null,THE,PRIVILEGED,ADMINISTRATOR,Mr.,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,2019-01-26 17:45:08.416,2019-01-26 17:45:08.362,2019-01-26 17:45:08.416,2019-01-26 17:45:08.362
ltdadmin,null,Limited,null,Administrator,Mr.,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,2019-01-26 17:45:08.46,2019-01-26 17:45:08.362,2019-01-26 17:45:08.46,2019-01-26 17:45:08.362
ltdadmin1,null,Limited,null,Administrator1,Mr.,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,2019-01-26 17:45:08.488,2019-01-26 17:45:08.362,2019-01-26 17:45:08.488,2019-01-26 17:45:08.362


In [13]:
val sql="""select first_name, last_name from ofbiz.person"""
val dataframe_mysql = sqlcontext.read.format("jdbc")
    .option("url", "jdbc:mysql://127.0.0.1:4000/ofbiz")
    .option("driver", "com.mysql.jdbc.Driver")
    .option("dbtable", s"( $sql ) t")
    .option("user", "ofbiz")
    .option("password", "ofbiz")
    .load()


Sat Jan 26 20:06:22 CST 2019 WARN: Establishing SSL connection without server's identity verification is not recommended. According to MySQL 5.5.45+, 5.6.26+ and 5.7.6+ requirements SSL connection must be established by default if explicit option isn't set. For compliance with existing applications not using SSL the verifyServerCertificate property is set to 'false'. You need either to explicitly disable SSL by setting useSSL=false, or set useSSL=true and provide truststore for server certificate verification.


sql = select first_name, last_name from ofbiz.person
dataframe_mysql = [first_name: string, last_name: string]


lastException: Throwable = null


[first_name: string, last_name: string]

In [14]:
%%dataframe --limit=5
dataframe_mysql

first_name,last_name
System,Account
null,null
THE,ADMINISTRATOR
Limited,Administrator
Limited,Administrator1


In [1]:
// connect to local mysql server
val sql="""select first_name, last_name from ofbiz.person"""
val dataframe_mysql = sqlcontext.read.format("jdbc")
    .option("url", "jdbc:mysql://127.0.0.1:3306/ofbiz")
    .option("driver", "com.mysql.jdbc.Driver")
    .option("dbtable", s"( $sql ) t")
    .option("user", "ofbiz")
    .option("password", "ofbiz")
    .load()

Sat Jan 26 21:17:45 CST 2019 WARN: Establishing SSL connection without server's identity verification is not recommended. According to MySQL 5.5.45+, 5.6.26+ and 5.7.6+ requirements SSL connection must be established by default if explicit option isn't set. For compliance with existing applications not using SSL the verifyServerCertificate property is set to 'false'. You need either to explicitly disable SSL by setting useSSL=false, or set useSSL=true and provide truststore for server certificate verification.


sql = select first_name, last_name from ofbiz.person
dataframe_mysql = [first_name: string, last_name: string]


[first_name: string, last_name: string]

In [2]:
%%dataframe --limit=5
dataframe_mysql

first_name,last_name
Accounting,Administrator
Acct,Buyer
THE,ADMINISTRATOR
Robert,Smith
Big,Al
